In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Imports

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
#from tensorflow.keras import backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.legacy import Adam
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.optimizers.legacy import Adam
from sklearn import datasets, metrics, svm

#Deal with warnings
import warnings
warnings.simplefilter('ignore')

#fast.ai library
#from fastai import *
#from fastai.vision import *



print("Libraries imported.")

This Kaggle competition project includes: Deep Learning, Convolution Classifier, Convolution Classifier with Batch Normalization and Dropout to improve training and Custom Convnets. I also used Perceptron but finally I commented it because it was to slow. I used softmax method to assess score and Adam optimizer to compile models (based on models scores). I attach links to sources which was really helpful to solving my problems and I really appreciate and thankful for authors of these solutions, methods.

I commented models with low scores. But I keep it in my project to show my project I was working on.

In [ ]:
train_data = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test_data = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
print("Dataset read.")

train_data.head()

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
test_data.head()

Max() function to asses imporant value for data normalization.

In [ ]:
train_data.to_numpy().max()

In [ ]:
test_data.to_numpy().max()

It is important value to data normalization. 

In [ ]:
#Encoding labels  - We encoded labels in 10 dimensions - because we have
#numbers between 0 - 9
#https://towardsdatascience.com/how-to-classify-handwritten-digits-in-python-7706b1ab93a3
labels = np.array(train_data.iloc[:,0])
encoding = []
for i in range(len(labels)):
    labels_zeros = [0,0,0,0,0,0,0,0,0,0]
    labels_zeros[labels[i]] = 1

encoding.append(labels_zeros)

In [ ]:
#Encoding labels  - We encoded labels in 10 dimensions - because we have
#numbers between 0 - 9
#https://towardsdatascience.com/how-to-classify-handwritten-digits-in-python-7706b1ab93a3
labels_test = np.array(test_data.iloc[:,0])
encoding_test = []
for i in range(len(labels)):
    labels_zeros_test = [0,0,0,0,0,0,0,0,0,0]
    labels_zeros_test[labels[i]] = 1

encoding_test.append(labels_zeros_test)

In [ ]:
test_data.values.reshape(28000, 784)

In [ ]:
test_data.shape

In [ ]:
X = train_data.drop('label', axis = 1)
y = train_data['label']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)

In [ ]:
#Data Normalization - so that the average is around 0
X_train = X_train / 255
X_test = X_test / 255



In [ ]:
X_train = X_train.values.reshape(-1,28,28,1)
X_test = X_test.values.reshape(-1,28,28,1)

In [ ]:
#X_test = X_test.as_matrix().reshape(28000, 784)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
# Take a look at what the images look like
rand_int = np.random.randint(0,40000)
img = X_train[rand_int].reshape(28,28)
plt.imshow(img, cmap = 'gray')

Perceptron (commented because it was to slow and less effective for this case)


#Standard Scaler

standard = StandardScaler()
standard.fit(X_train)

X_train_standared = standard.transform(X_train)
X_test_standared = standard.transform(X_test)


from sklearn.linear_model import Perceptron

#perceptron initialization
ppn = Perceptron(n_iter = 100,eta0=0.1,random_state=0)

#fit the model with standardized data
ppn.fit(X_train_standared,y_train)

#make predications
y_pred = ppn.predict(X_test_standared)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred) * 100
error = (1-accuracy_score(y_test, y_pred))*100


print("Accuracy: {} %".format(accuracy))
print("error: {} %".format(error))

Simple Sequential Models

The Convolutional Classifier

I choose softmax beacause this is not binary classification and adam optimizer, because sgd gave me worse results.

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)

In [ ]:
model0 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28,28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300, activation = 'relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(200, activation = 'relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(125, activation = 'relu'),
    tf.keras.layers.Dense(60, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation = 'softmax'),
])

#My first base model
model0 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28,28]),
    tf.keras.layers.Dense(300, activation = 'relu'),
    tf.keras.layers.Dense(300, activation = 'relu'),
    tf.keras.layers.Dense(200, activation = 'relu'),
    tf.keras.layers.Dense(10, activation = 'softmax'),
])

In [ ]:
model0.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model0.fit(X_train, y_train, epochs = 100, batch_size = 128, callbacks=[early_stopping],
                     validation_data = (X_test, y_test))

In [ ]:
pd.DataFrame(history.history).plot(
    figsize=(8,5), grid = True, xlabel = "Epoch")
plt.show()

In [ ]:
model0.evaluate(X_test, y_test)

The Convolutional Classifier with Dropout and Batch Normalization to improve training,

In [ ]:
model01 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28,28]),
    tf.keras.layers.Dense(300, activation = 'relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(300, activation = 'relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(200, activation = 'relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation = 'softmax'),
])

In [ ]:
model01.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history1 = model01.fit(X_train, y_train, epochs = 30, batch_size = 128, callbacks=[early_stopping],
                     validation_data = (X_test, y_test))

In [ ]:
pd.DataFrame(history1.history).plot(
    figsize=(8,5), grid = True, xlabel = "Epoch")
plt.show()

In [ ]:
model01.evaluate(X_test, y_test)

Underfitting improved.

Accuracy of data (both trained and tested set) are raise slowly while data was trained. Simultaneosly loss decreases as the data is trained so it is desired situation.

Convolutional Classifier with Batch Normalization and Dropout - to improve training and deal with overfitting.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model2 = keras.Sequential([
    layers.Flatten(input_shape=[28,28]),
    layers.Dense(200, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(100, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(10, activation = 'softmax'),
])

print("Model settings.")

In [ ]:
model2.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history2 = model2.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size = 128,
    epochs=60,
    callbacks=[early_stopping]
)

In [ ]:
pd.DataFrame(history2.history).plot(
    figsize=(8,5), grid = True, xlabel = "Epoch")
plt.show()

In [ ]:
model2.evaluate(X_test, y_test)

Custom Convnets

model_Convolution = tf.keras.models.Sequential()

model_Convolution.add(tf.keras.layers.Convolution2D(
    input_shape=(28,28,1),
    kernel_size=3,
    filters=6,
    strides=1,
    activation=tf.keras.activations.relu,
    #kernel_initializer=tf.keras.initializers.VarianceScaling()
))

model_Convolution.add(tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2),
    strides=(2, 2)
))

model_Convolution.add(tf.keras.layers.Convolution2D(
    kernel_size=3,
    filters=12,
    strides=1,
    activation=tf.keras.activations.relu,
    #kernel_initializer=tf.keras.initializers.VarianceScaling()
))

model_Convolution.add(tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2),
    strides=(2, 2)
))

model_Convolution.add(tf.keras.layers.Flatten())

#model.add(tf.keras.layers.Dense(
 #   units=128,
#    activation=tf.keras.activations.relu
#));

model_Convolution.add(tf.keras.layers.Dropout(0.2))

model_Convolution.add(tf.keras.layers.Dense(
    units=10,
    activation=tf.keras.activations.softmax,
    #kernel_initializer=tf.keras.initializers.VarianceScaling()
))

tf.keras.utils.plot_model(
    model_Convolution,
    show_shapes=True,
    show_layer_names=True,
)

model_Convolution.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history3 = model_Convolution.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=64,
    epochs=20,
    #callbacks=[early_stopping]
)

pd.DataFrame(history3.history).plot(
    figsize=(8,5), grid = True, xlabel = "Epoch")
plt.show()

model_Convolution.evaluate(X_test, y_test)

In [ ]:
model_Convolution3 = tf.keras.models.Sequential()

model_Convolution3.add(tf.keras.layers.Convolution2D(
    input_shape=(28,28,1),
    kernel_size=3,
    filters=20,
    strides=1,
    padding='same',
    activation=tf.keras.activations.relu,
    #kernel_initializer=tf.keras.initializers.VarianceScaling()
))

model_Convolution3.add(tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2),
    strides=(1, 1),
    padding='same'
))

model_Convolution3.add(tf.keras.layers.Convolution2D(
    kernel_size=3,
    filters=12,
    strides=1,
    padding='same',
    activation=tf.keras.activations.relu,
    
))

model_Convolution3.add(tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2),
    strides=(1, 1),
    padding='same'
))

model_Convolution3.add(tf.keras.layers.Flatten())

model_Convolution3.add(tf.keras.layers.Dropout(0.3))

model_Convolution3.add(tf.keras.layers.Dense(
    units=10,
    activation=tf.keras.activations.softmax,
))

In [ ]:
model_Convolution3.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history4 = model_Convolution3.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=1024,
    epochs=40,
    callbacks=[early_stopping]
)

In [ ]:
pd.DataFrame(history4.history).plot(
    figsize=(8,5), grid = True, xlabel = "Epoch")
plt.show()

In [ ]:
model_Convolution3.evaluate(X_test, y_test)

The best result so far.

model_new = tf.keras.models.Sequential()

#model_augmentation.add(data_augmentation)

model_new.add(tf.keras.layers.Convolution2D(
    input_shape=(28,28,1),
    kernel_size=3,
    filters=20,
    strides=1,
    padding='same',
    activation=tf.keras.activations.relu,
    #kernel_initializer=tf.keras.initializers.VarianceScaling()
))
model_new.add(tf.keras.layers.BatchNormalization())
model_new.add(tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2),
    strides=(1, 1),
    padding='same'
))



model_new.add(tf.keras.layers.Convolution2D(
    kernel_size=3,
    filters=12,
    strides=1,
    padding='same',
    activation=tf.keras.activations.relu,
    #kernel_initializer=tf.keras.initializers.VarianceScaling()
))
model_new.add(tf.keras.layers.BatchNormalization())
model_new.add(tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2),
    strides=(1, 1),
    padding='same'
))
model_new.add(tf.keras.layers.Dropout(0.2))

model_new.add(tf.keras.layers.Dense(units=6,activation=tf.keras.activations.relu
))
model_new.add(tf.keras.layers.Flatten())

model_new.add(tf.keras.layers.Dense(
    units=10,
    activation=tf.keras.activations.softmax,
    #kernel_initializer=tf.keras.initializers.VarianceScaling()
))

In [ ]:
train_data.shape

import tensorflow as tf
from tensorflow.keras import layers, models
#from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from pathlib import Path

from tensorflow.keras import layers
#https://stackoverflow.com/questions/61864244/how-to-avoid-augmenting-data-in-validation-split-of-keras-imagedatagenerator

dataAug = ImageDataGenerator(
    rotation_range = 20,          
    zoom_range=0.2
)

aug = dataAug.flow(X_train.reshape(-1, 28, 28, 1))

fig, axes = plt.subplots(2, 8, figsize = (15, 4))
plt.suptitle('Changes', fontsize=20, y=1.05)
axes = axes.ravel()

for i in range(16):
    
    aug_img = next(aug)[0]
    axes[i].imshow(aug_img, cmap = 'gray')
    axes[i].axis('off')
    
plt.show()

In [ ]:
import tensorflow.keras.layers

In [ ]:
data_augmentation1 = tf.keras.Sequential([
  layers.RandomRotation(0.2),
  layers.RandomZoom(0.2)
])


I decided not included vertical and horizontal flip, because model can misunderstood 6 and 9.

model_augmentation1 = tf.keras.models.Sequential()

model_augmentation1.add(data_augmentation1)

model_augmentation1.add(tf.keras.layers.Convolution2D(
    input_shape=(28,28,1),
    kernel_size=3,
    filters=20,
    strides=1,
    padding='same',
    activation=tf.keras.activations.relu,
    #kernel_initializer=tf.keras.initializers.VarianceScaling()
))

model_augmentation1.add(tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2),
    strides=(1, 1),
    padding='same'
))

model_augmentation1.add(tf.keras.layers.Convolution2D(
    kernel_size=3,
    filters=12,
    strides=1,
    padding='same',
    activation=tf.keras.activations.relu,
    
))

model_augmentation1.add(tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2),
    strides=(1, 1),
    padding='same'
))

model_augmentation1.add(tf.keras.layers.Flatten())

model_augmentation1.add(tf.keras.layers.Dropout(0.3))

model_augmentation1.add(tf.keras.layers.Dense(
    units=10,
    activation=tf.keras.activations.softmax,
))

model_augmentation1.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history7 = model_augmentation1.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=1024,
    epochs=40,
    #callbacks=[early_stopping]
)

pd.DataFrame(history7.history).plot(
    figsize=(8,5), grid = True, xlabel = "Epoch")
plt.show()

model_augmentation1.evaluate(X_test, y_test)

X_test = X_test.reshape(-1, 784,28000)

In [ ]:
test_data.shape

In [ ]:
x_test_re = test_data.to_numpy().reshape(28000, 28, 28)

In [ ]:
x_test_re.shape

In [ ]:
results = model_Convolution3.predict(x_test_re)

results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [ ]:
print(results)

In [ ]:
results_frame2 = pd.DataFrame(results)
results_frame2

In [ ]:
results_frame2['ImageId'] = results_frame2.index + 1
results_frame2

In [ ]:
submission_df = results_frame2[['ImageId', 'Label']].copy()
submission_df

In [ ]:
submission_df.to_csv("submission.csv",index=False)
print(submission_df.head())


In [ ]:
submission_df.head()

In [ ]:
submission_df.shape